In [ ]:
!pip install tsia

     |████████████████████████████████| 2.5 MB 7.6 MB/s 


In [ ]:
!pip install python-igraph

     |████████████████████████████████| 3.2 MB 8.3 MB/s 
  Created wheel for python-igraph: filename=python_igraph-0.9.8-py3-none-any.whl size=9070 sha256=bdb44d3ac2ce3066a4b70c799c36fb0f50c19974a3b6a43f4cf2e06063cf3f9c
  Stored in directory: /root/.cache/pip/wheels/15/86/ef/b8bcdfbcb1c489771ad256c7cd1eb4971cdb7f3f670938b798
Successfully built python-igraph


In [ ]:
!pip install cairocffi

     |████████████████████████████████| 88 kB 4.7 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.3.0-py3-none-any.whl size=89667 sha256=fef67d7093f19ca70a821565fc8128be79a04a5d072dfed1d7a33d5a9558a068
  Stored in directory: /root/.cache/pip/wheels/4e/ca/e1/5c8a9692a27f639a07c949044bec943f26c81cd53d3805319f
Successfully built cairocffi


In [ ]:
import igraph
import cairocffi as cairo
from sklearn import datasets
from sklearn import linear_model
from sklearn import manifold
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KDTree
import numpy as np
from sklearn.neighbors import KDTree
import igraph

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPRegressor

n_samples = 100
COLORS = ["#377eb8", "#4daf4a", '#ff7f00', '#00CED1', '#008000', '#D2691E', '#A020F0', '#FF1493', '#8B0000', '#FFD700']
#cores = azul escuro, laranja claro, verde claro, ciano, verde escuro, laranja escuro, roxo, rosa, vermelho, amarelo

In [ ]:
names = ["Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

In [ ]:
classifiers = [
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
def Reduzirclasses(class1, class2, X_train, y_train, X_test, y_test):
  for digit in range(0,10):
    if (digit != class1) and (digit != class2):
      ind_class = np.where(y_train == digit)
      y_train = np.delete(y_train, ind_class)
      X_train = np.delete(X_train, ind_class, axis = 0)

      ind_class_test = np.where(y_test == digit)
      y_test = np.delete(y_test, ind_class_test)
      X_test = np.delete(X_test, ind_class_test, axis = 0)

  print("X_train shape", X_train.shape)
  print("y_train shape", y_train.shape)
  print("X_test shape", X_test.shape)
  print("y_test shape", y_test.shape)

  return (X_train, y_train), (X_test, y_test) 

In [ ]:
(X_train, y_train), (X_test, y_test)  = fashion_mnist.load_data()
class1 = 2
class2 = 4
digit = 0

#(X_train, y_train), (X_test, y_test)  = Reduzirexemplos(digit, X_train, y_train, X_test, y_test)

(X_train, y_train), (X_test, y_test)   = Reduzirclasses(class1, class2, X_train, y_train, X_test, y_test)

#(train_images, train_labels), (test_images, test_labels)  = Aumentarexemplos(X_train, y_train, X_test, y_test)

#train_images = train_images.reshape(60000,28,28,1)
#test_images  = test_images.reshape(10000,28,28,1)

n_X_train = len(X_train)
n_X_test = len(X_test)

X_train = X_train.reshape(n_X_train, 784)
X_test = X_test.reshape(n_X_test, 784)

4431872/4422102 [==============================] - 0s 0us/step
X_train shape (12000, 28, 28)
y_train shape (12000,)
X_test shape (2000, 28, 28)
y_test shape (2000,)


## Grafos


In [ ]:
# relative neighborhood graph - retorna a matriz de adjacencias do grafo
# V = {v_i}, E = {e_{i, j}, j é vizinho de i}
def rng(X, k):
    adj_matrix = np.zeros((X.shape[0], X.shape[0])).astype(np.uint32)
    tree = KDTree(X)
    _, indices = tree.query(X, k=k + 1)
    
    for i, neighbor_list in enumerate(indices):
        for neighbor in neighbor_list[1:]:
            adj_matrix[i, neighbor] = 1
            # TODO: talvez j não seja um dos k-vizinhos de i, mas a gente cria
            # a aresta e_{j, i} mesmo assim
            adj_matrix[neighbor, i] = 1
    return adj_matrix


In [ ]:
def adj_to_igraph(adj, props):
    g = igraph.Graph()

    num_verts = adj.shape[0]
    g.add_vertices(num_verts)
    if props is not None:
        for key in props:
            g.vs[key] = props[key]
    edges = []
    for i in range(num_verts):
        for j in range(i + 1, num_verts):
            if adj[i, j] == 1:
                edges.append((i, j))

    g.add_edges(edges)
    return g


In [ ]:
colors = [COLORS[v] for v in y_test]
X_rng = rng(X_test, 5)
X_labels = [str(i) for i in range(X_test.shape[0])]
X_props = {"labels": X_labels, "colors": colors}
n_graph = adj_to_igraph(X_rng, X_props)

In [ ]:
igraph.plot(n_graph,layout=n_graph.layout_auto(), vertex_size=5, bbox=(0, 0, 500, 500), vertex_color=n_graph.vs["colors"])

In [ ]:
# recebe uma matriz de adjacencias e uma lista de labels. remove arestas
# da matriz de adjacencias se os labels forem iguais
def remove_non_boundary_edges(adj_matrix, y):
    new_adj = adj_matrix.copy()
    for i in range(new_adj.shape[0]):
        for j in range(i + 1, new_adj.shape[0]):
            if new_adj[i, j] == 1 and y[i] == y[j]:
                new_adj[i, j] = 0
                new_adj[j, i] = 0
    return new_adj

#clf_lr = linear_model.LogisticRegression()
clf_lr = classifiers[5]
clf_lr.fit(X_train, y_train)
train_acc = clf_lr.score(X_train, y_train)
test_acc = clf_lr.score(X_test, y_test)
print("train acc: ", train_acc)
print("test acc: ", test_acc)


/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


train acc:  0.67425
test acc:  0.637


In [ ]:
X_boundary_adj = remove_non_boundary_edges(X_rng, clf_lr.predict(X_test))
# grafo só com elementos que são vizinhos de alguém com label diferente
# As props são iguais
b_graph = adj_to_igraph(X_boundary_adj, X_props)


In [ ]:
igraph.plot(b_graph,layout=b_graph.layout_auto(), vertex_size=5, bbox=(0, 0, 900, 900), vertex_color=b_graph.vs["colors"])

In [ ]:
# retorna nova matriz de adjacencias e indices dos pontos que ficaram
def remove_disconnected_pts(adj_matrix):
    keep_ids = []
    for i in range(len(adj_matrix)):
        if np.sum(adj_matrix[i, :]) != 0:
            keep_ids.append(i)
    
    new_adj = np.zeros((len(keep_ids), len(keep_ids))).astype(np.uint32)
    for i in range(len(adj_matrix)):
        for j in range(i + 1, len(adj_matrix)):
            if adj_matrix[i, j] == 1:
                new_i = keep_ids.index(i)
                new_j = keep_ids.index(j)
                new_adj[new_i, new_j] = 1
                new_adj[new_j, new_i] = 1
    return new_adj, keep_ids

X_boundary_adj2, keep_ids = remove_disconnected_pts(X_boundary_adj)
ids = [str(i) for i in keep_ids]
X_boundary2_props = {}
X_boundary2_props["labels"] = ids
X_boundary2_props["colors"] = colors = [COLORS[y_train[i]] for i in keep_ids]
b_graph2 = adj_to_igraph(X_boundary_adj2, X_boundary2_props)
out = igraph.plot(b_graph2,layout=b_graph2.layout_auto(), vertex_size=8, bbox=(0, 0, 500, 500),
            vertex_color=b_graph2.vs["colors"])
out.save("/content/drive/Shareddrives/TRI/Grafos/ori_"+ names[5] + ".png")

In [ ]:
# TODO encontrar ponto por bisseção - por enquanto só retorna o ponto no
# meio da aresta
def get_boundary_sample(p1, p2):
    return (p1 + p2)*0.5

# repete os passos anteriors
def boundary_graph(X, y, k):
    neighbors_adj = remove_non_boundary_edges(rng(X, k), y)
    
    vertices = []
    labels = []
    for i in range(len(neighbors_adj)):
        for j in range(i + 1, len(neighbors_adj)):
            if neighbors_adj[i, j] == 1:
                labels.append(f"{i} - {j}")
                vertices.append(get_boundary_sample(X[i], X[j]))

    # vertices contém a lista de pontos entre duas amostras vizinhas e
    # de labels diferentes
    vertices = np.array(vertices)
    boundary_graph_adj = rng(vertices, k)
    
    return adj_to_igraph(boundary_graph_adj, {"labels": labels})


db_graph = boundary_graph(X_test, clf_lr.predict(X_test), 5)

In [ ]:
igraph.plot(db_graph,layout=db_graph.layout_auto(), vertex_size=10, bbox=(0, 0, 450, 450))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#import community
import community.community_louvain as community_louvain
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

from matplotlib.colors import to_hex

# Useful constants definition
COLORMAP = 'jet'


In [ ]:
def compute_network_graph_statistics(graph, b_graph2, keep_ids, y_test):

    #####Statistics from igraph
    #Diameter
    diameter = b_graph2.diameter(directed=False)

    #Number of edges
    contlaranja = 0
    contverde = 0
    for i in keep_ids:
      if y_test[i] == 4:
        contlaranja = contlaranja + 1
      if y_test[i] == 2:
        contverde = contverde + 1

    #Average path length
    avg_path_length = b_graph2.average_path_length()

    #Centralidade de Grau (Degree Centrality)
    p1 = np.array(b_graph2.degree())
    p1_m = np.mean(p1)
    #print("Centralidade de Grau: ", p)
    t1 = p1.argsort()[-10:][::-1]
    #print("Elementos mais centrais", t)

    #Centralidade de Proximidade (Closeness Centrality)
    p2 = np.array(b_graph2.closeness())
    p2_m = np.mean(p2)
    #print("Centralidade de Proximidade: ", p)
    t2 = p2.argsort()[-10:][::-1]
    #print("Elementos mais centrais", t)

    #Centralidade de Intermediação (Betweenness Centrality)
    p3 = np.array(b_graph2.betweenness())
    p3_m = np.mean(p3)
    #print("Centralidade de Intermediação: ", p)
    t3 = p3.argsort()[-10:][::-1]
    #print("Elementos mais centrais", t)

    #Centralidade de Autovalor (Eigenvalue Centrality)
    p4 = np.array(b_graph2.evcent())
    p4_m = np.mean(p4)
    #print("Centralidade de Autovalor: ", p)
    t4 = p4.argsort()[-10:][::-1]
    #print("Elementos mais centrais", t)

    #####Statistics from networkx
        
    partitions = community_louvain.best_partition(graph)
    nb_partitions = len(set(partitions.values()))
    modularity = community_louvain.modularity(partitions, graph)
    #diameter = nx.diameter(graph)
    node_size = list(nx.clustering(graph, weight='weight').values())
    avg_clustering_coeff = np.array(node_size).mean()
    density = nx.density(graph)
    #avg_path_length = nx.average_shortest_path_length(graph, weight='weight', method='dijkstra')
    
    average_degree = nx.average_degree_connectivity(graph)
    average_degree = np.mean(list(average_degree.values()))
    avg_weighted_degree = nx.average_degree_connectivity(graph, weight='weight')
    avg_weighted_degree = np.mean(list(avg_weighted_degree.values()))
    
    statisticsv = [diameter, contverde, contlaranja, average_degree, avg_weighted_degree, avg_path_length,
    p1_m, p2_m, p3_m, p4_m, density, avg_clustering_coeff, modularity, nb_partitions, names[0], train_acc,
    test_acc] 

    statistics = {
        'Diameter': diameter,
        'Number of green edges': contverde,
        'Number of orange edges': contlaranja,
        'Average degree': average_degree,
        'Average weighted degree': avg_weighted_degree,
        'Average path length': avg_path_length,
        'Average Degree centrality': p1_m,
        #'More central elements': t1,
        'Average Closeness centrality': p2_m,
        #'More central elements': t2,
        'Average Betweenness centrality': p3_m,
        #'More central elements': t3,
        'Average Eigenvalue centrality': p4_m,
        #'More central elements': t4,
        'Density': density,
        'Average clustering coefficient': avg_clustering_coeff,
        'Modularity': modularity,
        'Partitions': nb_partitions,

        'classifier': names[5],
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc
    }
    
    return statisticsv

In [ ]:
import networkx
A = b_graph2.get_edgelist()
#G = networkx.DiGraph(A) # In case your graph is directed
G = networkx.Graph(A) # In case you graph is undirected

In [ ]:
clf1_ori = compute_network_graph_statistics(G, b_graph2, keep_ids, y_test)

In [ ]:
clf1_ori

[33,
 650,
 633,
 3.858267610646896,
 3.858267610646896,
 10.52196293187666,
 2.790335151987529,
 0.13605837221517916,
 5393.824629773967,
 0.02043874219730938,
 0.0021765484804894923,
 0.0,
 0.8363314191192532,
 43,
 'Decision Tree',
 0.8440833333333333,
 0.81]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
clf6_ori = []
for i in range(0, 100):
  #colors = [COLORS[v] for v in y_test]
  X_rng = rng(X_test, 5)
  #X_labels = [str(i) for i in range(X_test.shape[0])]
  #X_props = {"labels": X_labels, "colors": colors}
  #n_graph = adj_to_igraph(X_rng, X_props)

  clf_lr = classifiers[5]
  clf_lr.fit(X_train, y_train)
  train_acc = clf_lr.score(X_train, y_train)
  test_acc = clf_lr.score(X_test, y_test)

  X_boundary_adj = remove_non_boundary_edges(X_rng, clf_lr.predict(X_test))
  #b_graph = adj_to_igraph(X_boundary_adj, X_props)

  X_boundary_adj2, keep_ids = remove_disconnected_pts(X_boundary_adj)
  ids = [str(i) for i in keep_ids]
  X_boundary2_props = {}
  X_boundary2_props["labels"] = ids
  X_boundary2_props["colors"] = colors = [COLORS[y_train[i]] for i in keep_ids]
  b_graph2 = adj_to_igraph(X_boundary_adj2, X_boundary2_props)

  A = b_graph2.get_edgelist()
  G = networkx.Graph(A)

  clf6_ori.append(compute_network_graph_statistics(G, b_graph2, keep_ids, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/discriminan

In [ ]:
clf6o = pd.DataFrame(clf6_ori, columns=['Diameter',
        'Number of green edges',
        'Number of orange edges',
        'Average degree',
        'Average weighted degree',
        'Average path length',
        'Average Degree centrality',
        'Average Closeness centrality',
        'Average Betweenness centrality',
        'Average Eigenvalue centrality',
        'Density',
        'Average clustering coefficient',
        'Modularity',
        'Partitions',
        'classifier',
        'Train Accuracy',
        'Test Accuracy'])

In [ ]:
clf6o.to_csv('/content/drive/Shareddrives/TRI/Grafos/clf6o.csv')

In [ ]:
from lightning import Lightning
from numpy import random

lgn = Lightning(ipython=True, local=True)

mat = random.rand(100,100)
mat[mat < 0.75] = 0
group = (random.rand(500) * 5).astype('int')

lgn.adjacency(mat, group=group)


In [ ]:
from lightning import Lightning
lgn = Lightning(ipython=True, local=True)
lgn.adjacency(X_boundary_adj2)